In [64]:
import pyspark
import pandas as pd
import numpy as np

In [65]:
from pyspark.sql import SparkSession

In [66]:
spark=SparkSession.builder.appName('FinalTryProjectData').getOrCreate()

In [67]:
spark

In [80]:
from pyspark.sql.types import *
filepath=r'D:\masters\bigdata\coursework\AB_US_2020.csv'
airbnb_scheme = StructType([StructField('id', IntegerType(), True),
                             StructField('name', StringType(), True),
                             StructField('host_id', IntegerType(), True),
                             StructField('host_name', StringType(), True),
                             StructField('neighbourhood_group', StringType(), True),
                             StructField('neighbourhood', StringType(), True),
                             StructField('latitude', DecimalType(10,5), True),
                             StructField('longitude', DecimalType(10,5), True),
                             StructField('room_type', StringType(), True),
                             StructField('price', IntegerType(), True),
                             StructField('minimum_nights', IntegerType(), True),
                             StructField('number_of_reviews', IntegerType(), True),
                             StructField('last_review', StringType(), True),
                             StructField('reviews_per_month', DecimalType(4,2), True),
                             StructField('calculated_host_listings_count', IntegerType(), True),
                             StructField('availability_365', IntegerType(), True),
                             StructField('city', StringType(), True)])

df_airbnb = spark.read \
  .option('multiLine', 'True') \
  .option('escape', '"') \
  .option("mode", "DROPMALFORMED")\
  .csv(filepath, header=True, schema=airbnb_scheme)

In [81]:
df_airbnb=df_airbnb.dropna()

In [82]:
pd_df_airbnb=df_airbnb.toPandas()

In [83]:
pd_df_airbnb
pd_df_airbnb.name = pd_df_airbnb.name.astype(str)
pd_df_airbnb.name=pd_df_airbnb.name.str.lower()
pd_df_airbnb.name=pd_df_airbnb.name.replace(['-','@','$','&','!'],'',regex=True)

In [84]:
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 
pd_df_airbnb["unigrams"] = pd_df_airbnb["name"].apply(nltk.word_tokenize)
pd_df_airbnb["unigrams"] =pd_df_airbnb["unigrams"].apply(lambda x: [item for item in x if item not in stop_words])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zainh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [85]:
rslt=pd.Series(np.concatenate([x for x in pd_df_airbnb.unigrams])).value_counts()
rslt = pd.DataFrame({'ngrams': list(rslt.keys()),
                   'count': list(rslt[:])})
rslt=rslt[rslt.ngrams != '.' ]
rslt=rslt.head(20)
rslt1=rslt['ngrams'][1:20]
rslt1

1          room
2       private
3       bedroom
4     apartment
5        studio
6         beach
7          cozy
9          home
10            2
11            1
12          apt
13     spacious
14         near
15        house
16         view
17        ocean
18    beautiful
19       modern
20        condo
Name: ngrams, dtype: object

In [86]:
def inner(text):    
    for word in text.split(' '):
        for c in rslt1:
            if word==c:
               return c
    return 'Null'       
c1=[]
for text in pd_df_airbnb['name']:
    c1.append(inner(text))
pd_df_airbnb['NameCleaned']=c1
pd_df_airbnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85147 entries, 0 to 85146
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   id                              85147 non-null  int32 
 1   name                            85147 non-null  object
 2   host_id                         85147 non-null  int32 
 3   host_name                       85147 non-null  object
 4   neighbourhood_group             85147 non-null  object
 5   neighbourhood                   85147 non-null  object
 6   latitude                        85147 non-null  object
 7   longitude                       85147 non-null  object
 8   room_type                       85147 non-null  object
 9   price                           85147 non-null  int32 
 10  minimum_nights                  85147 non-null  int32 
 11  number_of_reviews               85147 non-null  int32 
 12  last_review                     85147 non-null

In [87]:
print(c1)

['Null', 'Null', 'studio', 'Null', 'Null', 'ocean', 'condo', 'Null', 'Null', 'studio', 'Null', 'ocean', 'beach', 'ocean', 'beach', 'condo', 'ocean', 'Null', 'Null', 'house', 'ocean', 'ocean', 'ocean', '2', 'beach', '2', 'ocean', 'Null', 'Null', 'studio', 'ocean', 'Null', 'Null', 'Null', '1', 'ocean', 'Null', 'Null', 'studio', 'condo', 'home', 'Null', 'Null', 'beautiful', 'studio', 'Null', 'Null', 'condo', '1', '2', 'Null', 'beautiful', 'room', 'condo', 'Null', 'Null', 'Null', 'Null', 'beach', 'Null', 'beach', 'condo', 'home', '1', 'studio', 'Null', 'private', 'Null', '2', 'beach', 'Null', 'beach', 'beach', 'ocean', 'ocean', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'beach', 'studio', 'Null', 'condo', 'Null', 'Null', 'spacious', 'condo', 'house', 'home', 'studio', '2', 'ocean', 'beach', 'Null', 'Null', 'home', '2', 'condo', 'beautiful', '1', 'studio', 'ocean', 'ocean', 'ocean', 'private', 'near', 'Null', 'spacious', 'Null', 'studio', 'beach', 'home', 'house', 'Null', 'Null', 'Null

In [88]:
pd_df_airbnb=pd_df_airbnb[(pd_df_airbnb['price']<1000)  ]

In [89]:
df_group = pd_df_airbnb.groupby('NameCleaned').mean()  
df_group.sort_values(by=['price'], ascending=False)

,id,host_id,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365
NameCleaned,,,,,,,
ocean,2.248560e+07,8.092730e+07,252.629065,5.956242,32.262947,43.035327,228.395825
beach,2.242692e+07,7.466086e+07,230.363024,9.660239,42.324457,23.812672,192.714723
home,2.371746e+07,9.401111e+07,209.391868,12.629863,35.775324,9.134595,161.631265
2,2.304860e+07,8.393644e+07,204.980181,11.263908,33.068498,15.046245,173.855702
condo,2.247567e+07,8.428030e+07,199.343675,8.401551,28.189141,59.899165,218.678998
view,2.099530e+07,7.738249e+07,191.178791,9.693252,40.916740,26.812445,195.256792
house,2.268595e+07,8.730210e+07,190.224818,11.158637,48.864720,5.519708,171.857421
modern,2.389887e+07,7.554053e+07,178.703301,12.545243,40.242330,8.625243,143.303301
Null,2.248023e+07,8.486811e+07,166.961886,10.019057,40.443717,11.943421,169.423871


In [90]:
pd_df_airbnb=pd_df_airbnb.drop(columns=['unigrams'])

In [91]:
import pyspark.pandas as sppd
pd_df_airbnb["last_review"] = sppd.to_datetime(pd_df_airbnb["last_review"])

In [93]:
pd_df_airbnb.to_csv(r'C:\Users\zainh\Downloads\airbnbNLPfinaltry.csv',index=False)

In [94]:
from pyspark.sql.types import *
filepath=r'C:\Users\zainh\Downloads\airbnbNLPfinaltry.csv'
airbnb_scheme = StructType([StructField('id', IntegerType(), True),
                             StructField('name', StringType(), True),
                             StructField('host_id', IntegerType(), True),
                             StructField('host_name', StringType(), True),
                             StructField('neighbourhood_group', StringType(), True),
                             StructField('neighbourhood', StringType(), True),
                             StructField('latitude', DecimalType(10,5), True),
                             StructField('longitude', DecimalType(10,5), True),
                             StructField('room_type', StringType(), True),
                             StructField('price', DecimalType(10,5), True),
                             StructField('minimum_nights', IntegerType(), True),
                             StructField('number_of_reviews', IntegerType(), True),
                             StructField('last_review', StringType(), True),
                             StructField('reviews_per_month', DecimalType(4,2), True),
                             StructField('calculated_host_listings_count', IntegerType(), True),
                             StructField('availability_365', IntegerType(), True),
                             StructField('city', StringType(), True),
                             StructField('NameCleaned', StringType(), True),])

df_airbnb = spark.read \
  .option('multiLine', 'True') \
  .option('escape', '"') \
  .option("mode", "DROPMALFORMED")\
  .csv(filepath, header=True, schema=airbnb_scheme)

In [95]:
df_airbnb.show()

+-----+--------------------+-------+-----------------+-------------------+-----------------+--------+----------+---------------+---------+--------------+-----------------+-----------+-----------------+------------------------------+----------------+------+-----------+
|   id|                name|host_id|        host_name|neighbourhood_group|    neighbourhood|latitude| longitude|      room_type|    price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|  city|NameCleaned|
+-----+--------------------+-------+-----------------+-------------------+-----------------+--------+----------+---------------+---------+--------------+-----------------+-----------+-----------------+------------------------------+----------------+------+-----------+
| 5065|            mauka bb|   7257|            Wayne|             Hawaii|          Hamakua|20.04095|-155.43251|Entire home/apt| 85.00000|             2|               42| 2020-03-22|          

In [96]:
df_airbnb.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: decimal(10,5) (nullable = true)
 |-- longitude: decimal(10,5) (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: decimal(10,5) (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: decimal(4,2) (nullable = true)
 |-- calculated_host_listings_count: integer (nullable = true)
 |-- availability_365: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- NameCleaned: string (nullable = true)



In [97]:
from pyspark.sql.functions import *
df_airbnb=df_airbnb.withColumn("LastReview", to_date(df_airbnb.last_review))

In [98]:
df_airbnb=df_airbnb.withColumn("Days", datediff(current_date(),col("LastReview")))

In [99]:
df_airbnb.groupBy("NameCleaned").mean().toPandas()

,NameCleaned,avg(id),avg(host_id),avg(latitude),avg(longitude),avg(price),avg(minimum_nights),avg(number_of_reviews),avg(reviews_per_month),avg(calculated_host_listings_count),avg(availability_365),avg(Days)
0,apt,1.919089e+07,6.240918e+07,39.587320174,-85.104852127,147.555852843,11.201338,32.208027,0.918816,5.672241,123.718395,1181.494314
1,bedroom,2.253904e+07,8.198768e+07,37.382880749,-94.871235073,142.985021166,9.334093,27.521329,0.888444,8.179420,133.537936,1117.055031
2,beach,2.242692e+07,7.466086e+07,28.579235005,-132.950470003,230.363024181,9.660239,42.324457,1.252250,23.812672,192.714723,919.805632
3,view,2.099530e+07,7.738249e+07,32.269240359,-126.996721096,191.178790535,9.693252,40.916740,1.171157,26.812445,195.256792,966.059597
4,room,2.268703e+07,9.079580e+07,38.398943219,-90.879595843,76.193184151,8.071033,33.724697,1.034420,5.832273,147.655512,1098.277356
5,private,2.245302e+07,8.369441e+07,37.356988902,-98.501956015,103.383873100,9.126371,46.896497,1.380140,4.860410,140.207799,1018.903767
6,studio,2.204193e+07,7.718760e+07,35.523768182,-105.941324072,122.348422827,12.647002,41.568745,1.259612,13.549242,154.907055,1040.211173
7,condo,2.247567e+07,8.428030e+07,26.289283115,-142.258150621,199.343675418,8.401551,28.189141,0.888103,59.899165,218.678998,939.781026
8,spacious,2.149546e+07,7.259511e+07,38.480837871,-91.345359689,140.436021659,9.701054,29.078085,0.906039,7.217156,115.635794,1146.279282
9,1,2.079790e+07,7.146929e+07,36.320136364,-99.803050563,141.707334029,11.971846,28.180049,0.843410,12.191867,143.473410,1131.068474


In [100]:
#cols = df_airbnb.columns

In [112]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler,StandardScaler, MinMaxScaler
#categorical_indexer = StringIndexer(inputCols==["room_type","city"], outputCols=["roomtypeIndex","CityIndex"])
#onehotencoder_gender_vector = OneHotEncoder(inputCol="genderIndex", outputCol="gender_vec")

categoricalColumns = ['neighbourhood_group','room_type','city','NameCleaned']
stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]
numericCols = ['minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365','Days']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]
#scalar = MinMaxScaler(inputCol="features", outputCol="normFeatures")
#stages += [scalar]
print(stages)

[StringIndexer_1fa8c4f83b51, OneHotEncoder_44408b4a068b, StringIndexer_9f1b63ee2a67, OneHotEncoder_5c2a9977e493, StringIndexer_c810a74238c2, OneHotEncoder_528d15e4aed9, StringIndexer_78fc5847a221, OneHotEncoder_d8c2776a2a30, VectorAssembler_feccf54f1e42]


In [113]:
cols = df_airbnb.columns

In [114]:
print(cols)

['id', 'name', 'host_id', 'host_name', 'neighbourhood_group', 'neighbourhood', 'latitude', 'longitude', 'room_type', 'price', 'minimum_nights', 'number_of_reviews', 'last_review', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365', 'city', 'NameCleaned', 'LastReview', 'Days']


In [115]:
df_airbnb.toPandas()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,city,NameCleaned,LastReview,Days
0,5065,mauka bb,7257,Wayne,Hawaii,Hamakua,20.04095,-155.43251,Entire home/apt,85.00000,2,42,2020-03-22,0.45,2,365,Hawaii,Null,2020-03-22,822
1,5269,upcountry hospitality in the 'auwai suite,7620,Lea & Pat,Hawaii,South Kohala,20.02740,-155.70200,Entire home/apt,124.00000,30,10,2020-01-03,0.09,5,261,Hawaii,Null,2020-01-03,901
2,5387,hale koa studio 1 bedroom units,7878,Edward,Hawaii,South Kona,19.43119,-155.88079,Entire home/apt,85.00000,5,168,2020-03-18,1.30,3,242,Hawaii,studio,2020-03-18,826
3,5389,keauhou villa,7878,Edward,Hawaii,North Kona,19.56413,-155.96347,Entire home/apt,239.00000,6,20,2020-03-22,0.24,3,287,Hawaii,Null,2020-03-22,822
4,5390,stay at prince kuhio,7887,Todd,Kauai,Koloa-Poipu,21.88305,-159.47372,Entire home/apt,92.00000,3,143,2020-10-08,1.03,1,116,Hawaii,Null,2020-10-08,622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84236,43531859,chic seattle apartment near greenlake,6601753,Isabel,Other neighborhoods,Roosevelt,47.68457,-122.31550,Entire home/apt,100.00000,3,1,2020-06-06,1.00,1,99,Seattle,apartment,2020-06-06,746
84237,43554835,sl6 private¬∑modern¬∑quality¬∑convenient¬∑n s...,287025852,Li,Lake City,Olympic Hills,47.73462,-122.29509,Private room,79.00000,1,2,2020-10-06,2.00,3,337,Seattle,Null,2020-10-06,624
84238,43554849,"sl4 modern,private,quality,convenient,n seattle",287025852,Li,Lake City,Olympic Hills,47.73357,-122.29651,Entire home/apt,79.00000,1,1,2020-06-13,1.00,3,351,Seattle,Null,2020-06-13,739
84239,43589616,perfect spot for group events or family stays,347974040,Marcus,Delridge,Highland Park,47.51284,-122.33587,Entire home/apt,200.00000,1,2,2020-06-14,2.00,1,330,Seattle,Null,2020-06-14,738


In [117]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df_airbnb)
df_prepared = pipelineModel.transform(df_airbnb)
selectedCols = ["features"] + cols
df_prepared = df_prepared.select(selectedCols)
df_prepared.printSchema()

root
 |-- features: vector (nullable = true)
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: decimal(10,5) (nullable = true)
 |-- longitude: decimal(10,5) (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: decimal(10,5) (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: decimal(4,2) (nullable = true)
 |-- calculated_host_listings_count: integer (nullable = true)
 |-- availability_365: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- NameCleaned: string (nullable = true)
 |-- LastReview: date (nullable = true)
 |-- Days: integer (nullable = true)



In [118]:
df_prepared.toPandas().transpose()

,0,1,2,3,4,5,6,7,8,9,...,84231,84232,84233,84234,84235,84236,84237,84238,84239,84240
features,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",...,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
id,5065,5269,5387,5389,5390,5434,7896,8833,9877,13238,...,43304238,43437497,43438308,43461448,43498717,43531859,43554835,43554849,43589616,43603849
name,mauka bb,upcountry hospitality in the 'auwai suite,hale koa studio 1 bedroom units,keauhou villa,stay at prince kuhio,kauai ocean view 3 story tiki tower princeville,beachfront maui sunset condo,stay in the heart of kona,keolamauloa homestead,studio on princeville golf course / paniolo c...,...,perfect one bedroom in quiet neighborhood,mandala room 10 min walk to downtown,"the greek island room, 10 min drive to downtown",gorgeous lake washington view and special price,tranquil west seattle guesthouse with backyard,chic seattle apartment near greenlake,sl6 private¬∑modern¬∑quality¬∑convenient¬∑n s...,"sl4 modern,private,quality,convenient,n seattle",perfect spot for group events or family stays,rooftop tent in the heart of seattle
host_id,7257,7620,7878,7878,7887,7984,21844,21349,33179,51647,...,198922750,258796003,258796003,34956782,9295321,6601753,287025852,287025852,347974040,51816582
host_name,Wayne,Lea & Pat,Edward,Edward,Todd,Ahlea,Caroline,Noah,Kaye,Annie/Greg,...,Joseph,Dario,Dario,Starla,Michael,Isabel,Li,Li,Marcus,Fei
neighbourhood_group,Hawaii,Hawaii,Hawaii,Hawaii,Kauai,Kauai,Maui,Hawaii,Hawaii,Kauai,...,Capitol Hill,Capitol Hill,University District,Rainier Valley,West Seattle,Other neighborhoods,Lake City,Lake City,Delridge,Queen Anne
neighbourhood,Hamakua,South Kohala,South Kona,North Kona,Koloa-Poipu,North Shore Kauai,Kihei-Makena,North Kona,Hamakua,North Shore Kauai,...,Stevens,Broadway,University District,Rainier Beach,Fauntleroy,Roosevelt,Olympic Hills,Olympic Hills,Highland Park,East Queen Anne
latitude,20.04095,20.02740,19.43119,19.56413,21.88305,22.21972,20.75470,19.60709,20.02889,22.21961,...,47.62034,47.61497,47.66335,47.51301,47.52617,47.68457,47.73462,47.73357,47.51284,47.63913
longitude,-155.43251,-155.70200,-155.88079,-155.96347,-159.47372,-159.46958,-156.45666,-155.97424,-155.40747,-159.46998,...,-122.30779,-122.31664,-122.31106,-122.25115,-122.38116,-122.31550,-122.29509,-122.29651,-122.33587,-122.34293
room_type,Entire home/apt,Entire home/apt,Entire home/apt,Entire home/apt,Entire home/apt,Entire home/apt,Entire home/apt,Entire home/apt,Entire home/apt,Entire home/apt,...,Entire home/apt,Private room,Private room,Entire home/apt,Entire home/apt,Entire home/apt,Private room,Entire home/apt,Entire home/apt,Private room


In [119]:
dftrain, dftest = df_prepared.randomSplit([0.7, 0.3])
print("Training Dataset Count: " + str(dftrain.count()))
print("Test Dataset Count: " + str(dftest.count()))

Training Dataset Count: 59101
Test Dataset Count: 25140


In [121]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol="price", maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(dftrain)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [17.31791767040997,-16.279021290606916,1.040286943024016,-3.786530955811887,48.8238247464793,-2.810978677069258,-23.423015281156115,-7.018131788668392,62.97881255260832,-4.667583115683695,-14.619675123151778,130.53658769302302,-26.975983144116416,-10.81540149059663,81.5044174236369,67.58038843915025,0.17090672960047754,1.1646125692477052,35.44084240846459,-21.215815437218172,0.0,-11.018732547879006,-38.476757780168086,-15.413775702483589,-23.638618107476994,-26.259833969673746,0.0,-26.400603223663374,0.41579098116255575,-9.184147651339936,-6.07959362450645,45.966308578224236,-22.002059003685908,33.55384262127714,-60.523256221872,-95.99183971934266,-1.8619019864157413,0.0,-0.0,-6.368194317937081,4.513520888230447,-12.019028370122616,-32.88796342442592,-12.5562697083035,-58.876810395807496,-9.938620537466901,38.70851091071223,2.853081064851021,-25.690804861910756,-0.1539532686555541,-33.21231099909459,21.148642064608314,49.50054477228768,14.43381104702896,42.66607669943747,

In [122]:
dftrain.columns

['features',
 'id',
 'name',
 'host_id',
 'host_name',
 'neighbourhood_group',
 'neighbourhood',
 'latitude',
 'longitude',
 'room_type',
 'price',
 'minimum_nights',
 'number_of_reviews',
 'last_review',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365',
 'city',
 'NameCleaned',
 'LastReview',
 'Days']

In [123]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)
#print("pvalues",trainingSummary.pValues)

RMSE: 114.440893
r2: 0.265294


In [125]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
rf=RandomForestRegressor(featuresCol='features',labelCol='price',numTrees=100)
rf_model=rf.fit(dftrain)
rf_predictions = rf_model.transform(dftrain)
rf_evaluator = RegressionEvaluator(
    labelCol="price", predictionCol="prediction", metricName="rmse")
rmse = rf_evaluator.evaluate(rf_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 115.022
